# Filter out looses from the data (and second after)

In [1]:
# Average fps is 30
min_sequence_size = 30
delete_frames_before_death = 45
delete_frames_after_death = 30

In [2]:
# Is dead check invariant
def is_dead(frame):
    return np.min(frame[0][1][0:8,0:30]) == 0 and np.var(frame[0][1][0:8,0:30]) > 4800 and \
           np.max(frame[0][1][0:8,0:30]) >= 248 or np.mean(frame[0][1][17:20, 15:20]) == 48.8
    
deaths = 0
deleted = 0
def process_data(data):
    global deleted, deaths
    processed = []
    d = is_dead(data[0])
    dur = 0
    spawn_i = 0
    for i in range(1, len(data)):
        dead = is_dead(data[i])
        # Detect death
        if (not d) and dead:
            d = True
        # Count duration of death
        elif d and dead:
            dur += 1
        # Detect spawn
        elif d and not dead:
            if dur > 10:
                deaths += 1
                if spawn_i == 0:
                    if i-dur-delete_frames_before_death > 0 + min_sequence_size:
                        processed.append(data[0:i+1-dur-delete_frames_before_death])
                else:
                    if i-spawn_i-dur-delete_frames_before_death-delete_frames_after_death > min_sequence_size:
                        processed.append(data[spawn_i+delete_frames_after_death:i+1-dur-delete_frames_before_death])           
                spawn_i = i
                
            dur = 0
            d = False
        elif i==len(data)-1:
            start_point = 0 if spawn_i==0 else spawn_i+delete_frames_after_death
            if i-start_point > min_sequence_size:
                processed.append(data[start_point:i+1])   
                
    frames_saved = sum([len(i) for i in processed])
    deleted += len(data) - frames_saved
    return processed, frames_saved

In [3]:
import numpy as np
import os

data_input_path = "D:\\Python\\Keras\\Wormax\\data\\"
data_output_path = "D:\\Python\\Keras\\Wormax\\data_prepared\\"
list_dir = list(filter(lambda x: os.path.isfile, os.listdir(data_input_path)))

frames_counter = 0
for i, file_name in enumerate(list_dir):
    print('Processing', file_name, '\t{}/{}'.format(i+1, len(list_dir)))
    data = np.load(data_input_path + file_name)
    processed, frames_saved = process_data(data)
    if len(processed) != 0:
        np.save(data_output_path + "prepared_frame-{}-{}.npy".format(frames_counter, frames_counter + frames_saved), processed)
        frames_counter += frames_saved

Processing frame-0-10000.npy 	1/43
Processing frame-10000-11033.npy 	2/43
Processing frame-108890-118890.npy 	3/43
Processing frame-11033-16850.npy 	4/43
Processing frame-118890-128890.npy 	5/43
Processing frame-128890-138890.npy 	6/43
Processing frame-138890-148890.npy 	7/43
Processing frame-148890-158890.npy 	8/43
Processing frame-158890-168890.npy 	9/43
Processing frame-16850-18890.npy 	10/43
Processing frame-168890-178890.npy 	11/43
Processing frame-178890-179289.npy 	12/43
Processing frame-179289-189289.npy 	13/43
Processing frame-18890-28890.npy 	14/43
Processing frame-189289-199289.npy 	15/43
Processing frame-199289-209289.npy 	16/43
Processing frame-209289-219289.npy 	17/43
Processing frame-219289-229289.npy 	18/43
Processing frame-229289-239289.npy 	19/43
Processing frame-239289-249289.npy 	20/43
Processing frame-249289-258235.npy 	21/43
Processing frame-258235-268235.npy 	22/43
Processing frame-268235-278235.npy 	23/43
Processing frame-278235-288235.npy 	24/43
Processing fram

In [5]:
print(deaths)
# 361 12

361
